In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.nn import Conv2d
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from utilities import Old_CNN_bw, Old_CNN_color
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

In [2]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

Using the GPU!


In [3]:
#"../../../shenhao/sat_image_nhts/code/data_process/"

import pickle
with open('../census_id_to_index.pickle', 'rb') as handle:
    census_id_to_index = pickle.load(handle)

In [4]:
def training(image_array, x_id_train, y_train, x_id_test, y_test, cnn_net, optimizer, criterion, row_name, col_name, n_epochs = 100, batch_size = 200):
#More memory efficient?

    #training
    train_losses = []
    test_losses = []

    train_accuracies = []
    test_accuracies = []

    validate_losses = []
    validate_accuracies = []

    validate_losses_running = []
    validate_accurate_running = []


    test_indices = [census_id_to_index["0" + str(int(id)) if len(str(int(id))) == 10 else str(int(id))] for id in x_id_test]
    x_test_images = image_array[test_indices,:,:,:]
    x_test_images_norm_torch = torch.as_tensor(x_test_images).float()/255.0
    
    
    y_train_torch = torch.as_tensor(y_train[:,0]).float()
    y_test_torch = torch.as_tensor(y_test[:,0]).float()
    
    
    print("Training started: ")

    for n_epoch in range(n_epoches):
        # create permutation for batch training
        # To-Do: add permutation for SGD...But it is slow.
        #permutation = torch.randperm(x_train_images_norm_torch.size()[0])
        print("epoch: ", n_epoch)
        for i in tqdm(range(0, x_id_train.shape[0], batch_size)):
            # clear gradients first (for each iteration!)!
            optim.zero_grad()
            # forward pass

            #Get the indices in the image array of this batch
            ids = x_id_train[i:i+batch_size]
            indices = [census_id_to_index["0" + str(int(id)) if len(str(int(id))) == 10 else str(int(id))] for id in ids]

            #convert to torch and normalize
            to_torch = torch.as_tensor(image_array[indices, :, :, :]).float()/255.0

            batch_x, batch_y = to_torch.to(device), y_train_torch[i:i+batch_size].to(device)

            batch_y_pred_train = cnn_net(batch_x)
            # loss 
            
            loss = criterion(batch_y_pred_train.squeeze(), batch_y)
            # compute gradients
            loss.backward()
            # one step optim
            optim.step()

        # eval training accuracy
        with torch.no_grad():
            # evaluate training loss/accuracy in batches


            train_losses_running = []
            train_accurate_running = []

            validate_losses_running = []
            validate_accurate_running = []


            for i in range(0, x_id_train.shape[0], batch_size):

                # load batch data
                ids = x_id_train[i:i+batch_size]
                indices = [census_id_to_index["0" + str(int(id)) if len(str(int(id))) == 10 else str(int(id))] for id in ids]

                batch_image_train_eval = torch.as_tensor(image_array[indices, :, :, :]).float()/255.0
                batch_y_train = y_train_torch[i:i+batch_size].to(device)
                y_batch_probs_train = cnn_net(batch_image_train_eval.to(device))

                # evaluate loss on current batch
                current_batch_size = batch_y_train.size()[0]
                loss_train = criterion(y_batch_probs_train.squeeze(), batch_y_train)
                train_losses_running.append(loss_train.item() * batch_size)

                # evaluate accuracy on current batch
                _, batch_predict_train = torch.max(y_batch_probs_train, axis = 1)
                accurate_pred_train = r2_score(y_true=batch_y_train.cpu(), y_pred=batch_predict_train.cpu())
                train_accurate_running.append(accurate_pred_train)


            # append full-epoch (ie, all-batches) results to training loss, accuracy tally
            train_losses.append(sum(train_losses_running)/x_id_train.shape[0]) # average loss per epoch
            train_accuracies.append(sum(train_accurate_running) / x_id_train.shape[0]) # average accuracy per epoch


            # evaluate validation accuracy/loss in batches
            for i in range(0, x_id_test.shape[0], batch_size):

                # load batch data

                ids = x_id_test[i:i+batch_size]
                indices = [census_id_to_index["0" + str(int(id)) if len(str(int(id))) == 10 else str(int(id))] for id in ids]


                batch_image_val_eval = torch.as_tensor(image_array[indices, :, :, :]).float()/255.0
                batch_y_val = y_test_torch[i:i+batch_size].to(device)
                y_batch_probs_val = cnn_net(batch_image_val_eval.to(device))

                # evaluate loss on current batch
                current_batch_size = batch_y_val.size()[0]
                loss_validate = criterion(y_batch_probs_val.squeeze(), batch_y_val)
                validate_losses_running.append(loss_validate.item()*batch_size)

                # evaluate accuracy on current batch
                _, batch_predict_val = torch.max(y_batch_probs_val, axis = 1)
                accurate_pred_val = r2_score(y_true=batch_y_val.cpu(), y_pred=batch_predict_val.cpu())
                validate_accurate_running.append(accurate_pred_val)


            # append full-epoch (ie, all-batches) results to training loss, accuracy tally
            validate_losses.append(sum(validate_losses_running)/x_id_test.shape[0]) # average loss per epoch
            validate_accuracies.append(sum(validate_accurate_running) / x_id_test.shape[0]) # average accuracy per epoch


            # print performance updates
            if n_epoch % 5 == 0:
                print("Printing accuracies")
                #_, preds = torch.max(cnn_net(x_test_images_norm_torch.to(device)), axis=1)
                #unique = preds.unique(sorted=True)
                print('Epoch {}: train loss: {}'.format(n_epoch, train_losses[-1]))
                print('Epoch {}: train accuracy: {}'.format(n_epoch, train_accuracies[-1]))
                #print('Epoch {}: prediction counts: {}'.format(n_epoch, [(i.item(), (preds==i).sum().item()) for i in unique] ))
                print('Epoch {}: validate loss: {}'.format(n_epoch, validate_losses[-1]))
                print('Epoch {}: validate accuracy: {}'.format(n_epoch, validate_accuracies[-1]))
            elif n_epoch == 24:
                print("Printing accuracies")
                #_, preds = torch.max(cnn_net(x_test_images_norm_torch.to(device)), axis=1)
                #unique = preds.unique(sorted=True)
                print('Epoch {}: train loss: {}'.format(n_epoch, train_losses[-1]))
                print('Epoch {}: train accuracy: {}'.format(n_epoch, train_accuracies[-1]))
                #print('Epoch {}: prediction counts: {}'.format(n_epoch, [(i.item(), (preds==i).sum().item()) for i in unique] ))
                print('Epoch {}: validate loss: {}'.format(n_epoch, validate_losses[-1]))
                print('Epoch {}: validate accuracy: {}'.format(n_epoch, validate_accuracies[-1]))
                
                for_plotting.append((row_name,col_name,train_losses,validate_losses,train_accuracies,validate_accuracies))
                
                
    return train_losses, validate_losses, train_accuracies, validate_accuracies
   



In [5]:
labels_train = pd.read_csv("encoded_avg_less_labels_train.csv")
labels_test = pd.read_csv("encoded_avg_less_labels_test.csv")

In [6]:
labels_train.head()

,CENSUS_ID,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
0,5131010301,1.545455,0.000000,0.181818,0.090909,0.272727,0.454545,0.000000,0.000000,0.454545,0.545455,0.0,0.0
1,48167722800,1.600000,0.100000,0.200000,0.200000,0.100000,0.400000,0.000000,1.000000,0.000000,0.000000,0.0,0.0
2,34033020300,3.000000,0.181818,0.181818,0.136364,0.045455,0.454545,0.090909,0.636364,0.000000,0.272727,0.0,0.0
3,48491020313,2.444444,0.185185,0.185185,0.111111,0.129630,0.388889,0.000000,0.388889,0.407407,0.203704,0.0,0.0
4,36111954800,1.484848,0.166667,0.166667,0.121212,0.060606,0.484848,0.181818,0.636364,0.136364,0.045455,0.0,0.0


In [7]:
labels_test.head()

,CENSUS_ID,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
0,13215011100,2.211538,0.250000,0.192308,0.076923,0.096154,0.384615,0.076923,0.307692,0.057692,0.519231,0.000000,0.038462
1,48157671002,1.818182,0.136364,0.272727,0.045455,0.159091,0.386364,0.000000,0.545455,0.363636,0.068182,0.022727,0.000000
2,6071010020,0.666667,0.333333,0.222222,0.111111,0.111111,0.222222,0.111111,0.222222,0.000000,0.444444,0.222222,0.000000
3,48201423201,2.000000,0.047619,0.190476,0.190476,0.047619,0.523810,0.190476,0.571429,0.238095,0.000000,0.000000,0.000000
4,36111951300,2.269565,0.295652,0.200000,0.104348,0.121739,0.278261,0.043478,0.321739,0.408696,0.156522,0.052174,0.017391


In [8]:
x_id_train = labels_train["CENSUS_ID"].values.reshape(-1,1)
print(x_id_train.shape)
x_id_test = labels_test["CENSUS_ID"].values.reshape(-1,1)
print(x_id_test.shape)


dependent_variables = list(labels_train.columns[1:])
print(dependent_variables)

(16581, 1)
(3317, 1)
['HHVEHCNT', 'TRIPPURP_0', 'TRIPPURP_1', 'TRIPPURP_2', 'TRIPPURP_3', 'TRIPPURP_4', 'TRPTRANS_1', 'TRPTRANS_2', 'TRPTRANS_3', 'TRPTRANS_4', 'TRPTRANS_5', 'TRPTRANS_6']


In [9]:
image_array_bw = np.load("image_array_bw.npy", mmap_mode = 'r+')
image_array_color = np.load("image_array_color.npy", mmap_mode = 'r+')

In [10]:
models = ["Resnet18_bw", "Resnet18_color"]

In [11]:
results = pd.DataFrame(columns=dependent_variables, index = models)
#[(row_name,col_name,train_losses,validate_losses,train_accuracies,validate_accuracies)]
for_plotting = []

In [12]:
results.head()

,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
Resnet18_bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Resnet18_color,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#%%capture output
for row_name in models:
    for col_name in dependent_variables:
        print("------")
        print(row_name, col_name)
        print("------")
        K = 1
        if row_name == "Old_CNN_bw":
            cnn_net = Old_CNN_bw(K).float().to(device)
            image_array = image_array_bw
            
        elif row_name == "Old_CNN_color":
            cnn_net = Old_CNN_color(K).float().to(device)
            image_array = image_array_color
            
        elif row_name == "Resnet18_bw":
            model_ft = torchvision.models.resnet18(pretrained=False)
            for param in model_ft.parameters():
                param.requires_grad = True
            model_ft.conv1 = Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            num_ftrs = model_ft.fc.in_features
            model_ft.fc = nn.Linear(num_ftrs, K, bias = True)
            cnn_net = model_ft.float().to(device)
            
            
            image_array = image_array_bw
            
        elif row_name == "Resnet18_color":
            model_ft = torchvision.models.resnet18(pretrained=False)
            num_ftrs = model_ft.fc.in_features
            model_ft.fc = nn.Linear(num_ftrs, K, bias = True)
            cnn_net = model_ft.float().to(device)
            image_array = image_array_color
        
        optim = torch.optim.Adam(cnn_net.parameters(), lr=0.01)
        criterion = nn.MSELoss()
        n_epoches = 25 # To-Do: need more epoches.
        batch_size = 200
        #K = 1 here because we are doing regression, so the final layer output should have size 1
        
        
        y_train = labels_train[col_name].values.reshape(-1,1)
        y_test = labels_test[col_name].values.reshape(-1,1)
        
        training(image_array, x_id_train, y_train, x_id_test, y_test, 
                 cnn_net, optim, criterion, row_name, col_name, n_epochs = n_epoches, batch_size = batch_size)
        print("*********")
        print(for_plotting)
        print("*********")
        
        #torch.cuda.empty_cache()
        

------
Resnet18_bw HHVEHCNT
------
Training started: 
epoch:  0



Printing accuracies
Epoch 0: train loss: 0.37136694848785484
Epoch 0: train accuracy: -0.05937194488868916
Epoch 0: validate loss: 0.39091009780301866
Epoch 0: validate accuracy: -0.05994948095653854
epoch:  1



epoch:  2



epoch:  3



epoch:  4


...


Printing accuracies
Epoch 20: train loss: 0.004826206930161896
Epoch 20: train accuracy: -0.0005040021026526038
Epoch 20: validate loss: 0.004628075206445073
Epoch 20: validate accuracy: -0.0004634916991154126
epoch:  21



epoch:  22



epoch:  23



epoch:  24



Printing accuracies
Epoch 24: train loss: 0.00473450563823754
Epoch 24: train accuracy: -0.0005040021026526038
Epoch 24: validate loss: 0.004385938840691659
Epoch 24: validate accuracy: -0.0004634916991154126
*********
[('Resnet18_bw', 'HHVEHCNT', [0.37136694848785484, 0.3593664169023943, 0.36272750272659404, 0.35782497809596275, 0.3702675521204795, 0.36374132485536664, 0.35680053130715167, 0.3671710557755296, 0.36309912713616055, 0.35738462950580696, 0.3605791559179674, 0.34997998449499146, 0.3498218831837116, 0.35717750219691685, 0.35991469228176365, 0.3604196210181021, 0.3561958685893862, 0.3505118969057489, 0.34772299697866316, 0.35851757644737065, 0.35614503011593635, 0.3524605025500859, 0.35611294296680607, 0.3537975183823638, 0.3507342670573651], [0.39091009780301866, 0.3808549361880155, 0.3844567404693245, 0.38038257521823976, 0.3944242974523683, 0.38761151171176234, 0.3823959148178124, 0.3929690995559945, 0.39069709879898473, 0.3815393922988068, 0.38498324516993315, 0.3790300

In [14]:
print(for_plotting)

[('Resnet18_bw', 'HHVEHCNT', [0.37136694848785484, 0.3593664169023943, 0.36272750272659404, 0.35782497809596275, 0.3702675521204795, 0.36374132485536664, 0.35680053130715167, 0.3671710557755296, 0.36309912713616055, 0.35738462950580696, 0.3605791559179674, 0.34997998449499146, 0.3498218831837116, 0.35717750219691685, 0.35991469228176365, 0.3604196210181021, 0.3561958685893862, 0.3505118969057489, 0.34772299697866316, 0.35851757644737065, 0.35614503011593635, 0.3524605025500859, 0.35611294296680607, 0.3537975183823638, 0.3507342670573651], [0.39091009780301866, 0.3808549361880155, 0.3844567404693245, 0.38038257521823976, 0.3944242974523683, 0.38761151171176234, 0.3823959148178124, 0.3929690995559945, 0.39069709879898473, 0.3815393922988068, 0.38498324516993315, 0.37903002776904104, 0.3805003918759033, 0.3915759447579072, 0.3952482988721533, 0.39695687311257016, 0.3854333455928449, 0.38402103716471053, 0.38790877842205923, 0.3869765416519952, 0.38764770935916526, 0.38945495920580914, 0.3